In [1]:
import torch

In [2]:
from deep_speaker_pytorch.triplet_loss import DeepSpeakerLoss
from deep_speaker_pytorch.batcher import DeepSpeakerTripletDataset,LazyTripletBatcher,KerasFormatConverter
from deep_speaker_pytorch.model import DeepSpeaker
from deep_speaker_pytorch.audio import Audio
from deep_speaker_pytorch.utils import train_test_sp_to_utt,load_pickle

In [3]:
import torch.nn as nn   
import torch.nn.functional as F
from collections import deque

In [3]:
model=DeepSpeaker()

In [4]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [9]:
class DIEMSimilarity(nn.Module):
    def __init__(self,v_m=-1,v_M=1):
        super(DIEMSimilarity,self).__init__()
        self.dq=deque(maxlen=1000)
        self.v_M=v_M
        self.v_m=v_m
        self.ed=None
        self.var=None
    
    def forward(self,batch1:torch.tensor,batch2:torch.tensor)->torch.tensor:
        for v1,v2 in zip(batch1,batch2):
            self.dq.append(v1.tolist())
            self.dq.append(v2.tolist())
        t=torch.tensor(list(self.dq))
        self.var=t.var()
        pdist=F.pdist(t)
        self.ed=pdist.mean()
        # self.v_M=t.max()
        # self.v_m=t.min()
        batch_diem=(self.v_M-self.v_m)*(torch.linalg.vector_norm(batch1-batch2,ord=2,axis=1)-self.ed)/self.var
        return torch.sum(batch_diem)

In [ ]:
audio=Audio(cache_dir='/mnt/d/Programs/Python/PW/projects/speech/embeddings/working_dir',
            audio_dir='/mnt/c/Users/rahim/Downloads/archive/data',
            sample_rate=16_000)

In [26]:
kc=KerasFormatConverter('/mnt/d/Programs/Python/PW/projects/speech/embeddings/working_dir')

In [27]:
kc.persist_to_disk()

In [30]:
categorical_speakers=load_pickle('/mnt/d/Programs/Python/PW/projects/speech/embeddings/working_dir/keras-inputs/categorical_speakers.pkl')

In [32]:
categorical_speakers.get_index('0001')

0

In [5]:
batcher=LazyTripletBatcher('/mnt/d/Programs/Python/PW/projects/speech/embeddings/working_dir',
                           160,model,device)

Initializing the batcher:   0%|          | 0/4 [00:00<?, ?it/s]/mnt/d/Programs/Python/PW/projects/speech/speech_env_torch/lib/python3.12/site-packages/deep_speaker_pytorch/batcher.py:129: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  model_inputs=torch.tensor(model_inputs,device=self.device).float()
Initializing the batcher: 100%|██████████| 4/4 [00:24<00:00,  6.07s/it]


In [6]:
batcher.get_random_batch(9)

(tensor([[[[-0.5870, -0.5787, -0.5728,  ..., -0.3908, -0.3095, -0.4094],
           [-0.5102, -0.5111, -0.4948,  ..., -0.4717, -0.4672, -0.4985],
           [-0.4439, -0.4402, -0.4240,  ..., -0.4165, -0.4145, -0.4322],
           ...,
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
 
 
         [[[-0.5457, -0.5456, -0.5456,  ...,  0.5854,  0.3795, -0.2786],
           [-0.5872, -0.5874, -0.5874,  ...,  0.7210,  1.4015, -0.2068],
           [-0.7530, -0.7513, -0.7539,  ...,  0.6355,  1.5747, -0.1326],
           ...,
           [-0.2306, -0.1067, -0.1617,  ..., -0.5332, -0.5346, -0.5438],
           [-0.6845, -0.6551, -0.5847,  ..., -0.6692, -0.6799, -0.6838],
           [-0.5502, -0.5156, -0.3960,  ..., -0.5562, -0.5666, -0.5686]]],
 
 
         [[[-0.0684,  2.9339,  0.0871,  ..., -0.0715, -0.0656, -0.4948],
       